In [ ]:
################################################################################
# Author 1:      Bernhard Lugger
# MatNr 1:       01114792
# Author 2:      Paula Nauta
# MatNr 2:       11938311
# File:          assignment2.ipynb
# Description: ... short description of the file ...
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [3]:
# imports
import matplotlib.pyplot as plt # or from matplotlib import pyplot as plt
import matplotlib.dates as mdates # we use this to space our date-xticks
import numpy as np
import pandas as pd

import plotly.express as px
# Plotly express is especcialy useful if you want to plot dataframes (e.g. Pandas Dataframe).
from plotly.subplots import make_subplots
# import plotly.graph_objects as go


In [ ]:
# funktionen 

def load_oecd_data(file_path):
    """loads one table at the given path and returns it to a dataframe."""
    #file_path_adv = r'athlete_events.csv'  # r before path to avoid '\' in path
    oecd_data = pd.read_csv(file_path, index_col=False)
    return oecd_data

def plot_lineplot(country1, country2, country3):
    
